# Feedback Controls
### Tutorial on Feedback Systems (part 1 of N)
##### R. X Adhikari (5/2020)

### Useful References
* [Feedback Controls by Astrom and Murray](http://www.cds.caltech.edu/~murray/amwiki/index.php/Second_Edition)
* [Why Null Measurements](https://courses.lumenlearning.com/physics/chapter/21-5-null-measurements/)
* [Gravitational Wave Detectors, Saulson (2017)](https://www.google.com/books/edition/_/5QVGDQEACAAJ?hl=en&sa=X&ved=2ahUKEwicmp_3v5TqAhUUJTQIHTdSDEoQre8FMAB6BAgAEA4)
* [Controls Tutorials](https://wiki.ligo.org/CSWG/Training_References) from the LIGO/Virgo/KAGRA Control Systems Working Group

In [ ]:
import sys,os
import numpy as np

from scipy import signal as sig
#from scipy import interpolate
#from scipy import optimize
import scipy.constants as const

# this is the Caltech py-controls package from Richard Murrary's group
# https://python-control.readthedocs.io/en/0.8.3/intro.html#installation
# Recommend installing in a dedicated Anaconda python environment:
# conda install -c conda-forge control
import control
import control.matlab as mat

from timeit import default_timer as timer  # this is for timing the ODE solvers
from IPython.display import Image, SVG, display

import matplotlib.pyplot as plt
plt.rcParams.update({'text.usetex': False,
                     'lines.linewidth': 4,
                     'font.family': 'serif',
                     'font.serif': 'Georgia',
                     'font.size': 22,
                     'xtick.direction': 'in',
                     'ytick.direction': 'in',
                     'xtick.labelsize': 'medium',
                     'ytick.labelsize': 'medium',
                     'axes.labelsize': 'medium',
                     'axes.titlesize': 'medium',
                     'axes.grid.axis': 'both',
                     'axes.grid.which': 'both',
                     'axes.grid': True,
                     'grid.color': 'xkcd:beige',
                     'grid.alpha': 0.253,
                     'lines.markersize': 12,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 'small',
                     'legend.framealpha': 0.8,
                     'legend.handletextpad': 0.5,
                     'legend.labelspacing': 0.33,
                     'legend.loc': 'best',
                     'figure.figsize': ((12, 8)),
                     'savefig.dpi': 140,
                     'savefig.bbox': 'tight',
                     'pdf.compression': 9})

# uncomment if you have a Mac with Retina display
#%config InlineBackend.figure_format = 'retina'

# this is a nice jupyter lab theme:
# jupyter labextension install @oriolmirosa/jupyterlab_materialdarker

def multZPG(P, C, A):
    # multiply them together by concatenating the zeros and poles
    pos = np.hstack((P.poles, C.poles, A.poles))
    zos = np.hstack((P.zeros, C.zeros, A.zeros))
    G = P.gain * C.gain * A.gain
    return sig.ZerosPolesGain(zos, pos, G)

## Outline
1. Why Feedback?
1. Linear Systems
1. Laplace Domain
1. Feedback Loops
    1. Nomenclature
    1. How to calculate TFs for SISO
    1. MIMO calculations
        1. Adjacency Matrix
1. Noise Analysis
1. Range Analysis
1. Stability Analysis
1. Optimal Feedback

<div class="alert alert-success">

# Why do we need Feedback Controls?
Within GW detectors, many of the systems are very weakly nonlinear, and feedback is useful to keep it linear. The interferometer itself has mainly nonlinear readbacks of the displacement degrees of freedom and so the feedback systems are used to keep it at an operating point which maximizes the sensitivity.

In experimental physics, there is a lot of discussions of the virtues of making 'null' measurements (e.g. the Wheatstone Bridge in electronics, the Michelson interferometer in displacement measurements). These are special cases of using feedback to achieve the optimum operating point. The experimenter manually adjusts the operating point to make a null measurement.

### some nuggets of wisdom about control systems
1. For linear systems, the application of noiseless feedback does not increase or decrease the sensitivity of that system to external disturbances. i.e. a GW detector, operating in the linear regime, has the same sensitivity whether or not the feedback gain is zero or infinity.
1. When using feedback, we have to make sure that the design respects [**Causality**](https://en.wikipedia.org/wiki/Kramers%E2%80%93Kronig_relations). i.e. we cannot have useful negative feedback if the delay in the loop is too large. This is encapsulated in the "phase margin" of Bode plots, but can be confusing to interperet if the phase delay exceeds 180 deg.
1. Practically, the causality condition limits how much gain one can have in the loop, or how much low pass filtering: there is a correspondence between the magnitude and phase of linear systems and that includes the feedback controllers that we design.
1. Feed *forward* (or "sensor correction*) is one way to accomplish some noise suppression without being limited by large delays in the physical plant.
    
</div>

## [Linear Systems](https://en.wikipedia.org/wiki/Linear_system)
[Linear Systems](https://en.wikipedia.org/wiki/Linear_system) are key for the understanding of linear, time-invariant (LTI) control systems. 

We understand that all physical systems are nonlinear at some level, but it is useful to analyze their linear (small signal) response. Some systems (e.g. the Maxwell's Equations for E&M) are very, very linear, while others (like the Pound-Drever-Hall signal of a Fabry-Perot cavity), are very nonlinear (for a high Finesse FP cavity, the linear regime is less than 0.1% of the phase space).

### Assumptions:
1. Scaling the input by a factor $\alpha$, will also scale the output by the factor $\alpha$
1. For time-invariant linear systems (LTI), the function that relates the inputs to the outputs does not change as a function of time. Real physical systems do change, but it is often useful to study their slowly drifting parameters by assuming that we can treat the system as an LTI system at each instance during its evolution.
1. If all of the inputs to the system are zero, then all of the outputs will eventually go to zero.
1. We will usually analyze these systems in the *frequency domain*. This is a very useful techique that allows us to design the feedback in a simple way, using just algebra, rather than use integrals to compute the input-output relations.


## Laplace Domain
It is common to analyze LTI systems in the Laplace or "s" domain.

## Feedback Loops
Recommend [Astrom and Murray (2009)](https://www.cds.caltech.edu/~murray/amwiki) for a nice introduction. Here lets define some terms to get started:
* **Open Loop Gain (OLG):** once around the open loop: $$OLG = P \times C \times A$$
* **Closed Loop Gain (CLG):** $$CLG = \frac{1}{1 - OLG}$$
* **Forward Loop Gain (FLG):** the open-loop gain between 2 points in the loop (e.g. between a signal injection point and the next readout port) 
* [Sensitivity](https://en.wikipedia.org/wiki/Sensitivity_(control_systems)): roughly speaking, this is the maximum peak in the CLG (i.e. how much "gain peaking" is there?)
* **Plant:** this is the physical system that we are trying to control (e.g. a the position of a pendulum, the temperature of a box, the frequency of a quartz oscillator, etc.
* **Actuator:** this is what we use to change the plant (e.g. a force, a heat source, a variable capacitor, etc.)
* **Controller:** this is where the magic happens - this is where the feedback loop transfer function is implemented. IF its digital, the number of poles & zeros here can be very large $\sim O(100)$
* **Error Signal ($\epsilon$):** the input to the feedback Controller $C(s)$
* **Control Signal ($u$):** the out of the feedback controller (e.g. the voltage that drives the actuator)
* [Dynamic Range](https://en.wikipedia.org/wiki/Dynamic_range): multiple definitions abound; one useful definition is that the dynamic range is the ration of (1) the largest possible signal a sensor can receive without 'saturating', to (2) the fundamental noise level of that sensor. For a low-noise opamp, the dynamic range is ~ (10 V)(1 nV/rHz) ~ $10^{11}$ for a 1 second measurement.

In [ ]:
SVG(filename='SISO-loop.svg')

# Example 1: Feedback Control of a Suspended Fabry-Perot Cavity
The Plant, P(s), includes the frequency response of the FP cavity:
$$ \frac{E_{\rm refl}}{x_{\rm mirror}}(\omega) = \frac{\rm DC~gain}{1 + i \omega/\omega_c}$$
as well as the pendulum motion of the mirror to an applied force:
$$ \frac{x(f)}{F(f)} = \frac{1/m}{\omega_0^2 + i \frac{\omega_0 \omega}{Q} - \omega^2}$$
where $\omega = 2 \pi f$ and $\omega_c$ is $2 \pi \times f_c$, the cavity pole frequency.

For this example, we 

In [ ]:
f = np.logspace(-1, 4, 500)
w = 2*np.pi * f

f_cav = 375 # cavity pole frequency [Hz]

# pendulum poles
f_pend = 1
p1 = 1 * np.exp(1j * 88*np.pi/180)
p_pend = -2*np.pi*np.array([p1, np.conj(p1)])
# Convert the zpk model into state-space
P = mat.zpk2ss([], p_pend, 1/m_mirror/(2*np.pi*f_pend)**2)
Pend = control.ss(P[0], P[1], P[2], P[3])

P = mat.zpk2ss([], [-2*np.pi*f_cav], 3e12)
Cavity = control.ss(P[0], P[1], P[2], P[3])

# the feedback controller
C = mat.zpk2ss([-2*np.pi*30, -2*np.pi*30], [0, -2*np.pi*1500], 1e5)
C = control.ss(C[0], C[1], C[2], C[3])

# The Actuator, A, is flat up to 1111 Hz, with a constant force coeff [Newtons/Volt]
A = mat.zpk2ss([],[-2*np.pi*1111],1)
A = control.ss(A[0], A[1], A[2], A[3])

# multiply them together 
olg = control.series(Pend, Cavity, C, A)

# unity gain feedback connection of the OLG to make CLG
clg = control.feedback(1, olg)

#w, mag, phase = sig.bode(olg, w=w)
mag_pend,   pha_pend,   _ = control.freqresp(Pend,   omega=w)
mag_cav,   pha_cav,   _ = control.freqresp(Cavity,   omega=w)
mag_c,   pha_c,   _ = control.freqresp(C,   omega=w)
mag_olg, pha_olg, _ = control.freqresp(olg, omega=w)
mag_clg, pha_clg, _ = control.freqresp(clg, omega=w)
mag_clg = mag_clg.flatten()
mag_olg = mag_olg.flatten()
pha_clg = pha_clg.flatten()
pha_olg = pha_olg.flatten()
mag_pend = mag_pend.flatten()
pha_pend = pha_pend.flatten()
mag_cav = mag_cav.flatten()
pha_cav = pha_cav.flatten()
mag_c = mag_c.flatten()
pha_c = pha_c.flatten()
#hclg = 1 / (1 - holg)

# plot the results
fig,ax = plt.subplots(2, 2, figsize=(19,10), sharex='col')

ax[0,0].semilogx(f, 20*np.log10(mag_pend * mag_cav), label='Plant', ls='--')
ax[0,0].semilogx(f, 20*np.log10(mag_c), label='Controller', ls='--')

ax[0,0].semilogx(f, 20*np.log10(mag_clg), label='CLG')
ax[0,0].semilogx(f, 20*np.log10(mag_olg), label='OLG')
ax[0,0].set_ylim([-60, 141])

ax[0,0].set_ylabel('Mag [dB]')
ax[0,0].set_xticklabels([])
ax[0,0].legend()

ax[1,0].semilogx(f, 180/np.pi*pha_pend, ls='--')
ax[1,0].semilogx(f, 180/np.pi*pha_c, ls='--')
ax[1,0].semilogx(f, 180/np.pi*pha_clg)
ax[1,0].semilogx(f, 180/np.pi*pha_olg)

ax[1,0].set_xlabel('Freq [Hz]')
ax[1,0].set_ylabel('Phase [deg]')
ax[1,0].set_ylim([-180, 181])
ax[1,0].set_yticks(np.arange(-180, 181, 60))

# now make the impulse and step response plots
#T = np.linspace(0, 0.051, 3000)
t, y = control.impulse_response(clg)
ax[0,1].plot(t, y, label='Impulse Response')
ax[0,1].legend()

t, y = control.step_response(clg)
ax[1,1].plot(t, y, label='Step Response')
ax[1,1].set_xlabel('Time [s]')
ax[1,1].legend()
plt.show()

# Get the gain and phase margins from the Open Loop Gain
gm, pm, wg, wp = control.margin(mag_olg, 180/np.pi*pha_olg, w)
#gm, pm, wg, wp = control.margin(olg)
print('Phase Margin = {:0.1f} deg.'.format(pm))
print('Gain Margin  = {:0.1f} dB.'.format(20*np.log10(gm)))
#K, CL, gam, rcond = control.hinfsyn(P, 1, 1)

## Noise Analysis
#### How to calculate the TF from the noise injection points to elsewhere in the system:
Simple - 
1. The loop suppresses the disturbance, $d$, so that right after the injection point: $$\textrm{in-loop noise} = d \times \frac{1}{1 - OLG}$$
1. From that point multiply by the forward loop gain between the injection point and the point of interest.

#### Example
* How big is the contribution of the force noise "Disturbance" in the error signal? After appling the CLG, we see that the only block between the injection of force noise and the error signal is the plant block, $P$:

$$\epsilon = d \times \frac{P}{1-OLG}$$ 

In the case of (quantum noise) for a single mirror in a Fabry-Perot cavity, we have:

$$F_{\rm RPN}(\omega) = \frac{2}{c} \sqrt{\frac{h c}{\lambda}\frac{P_{in}}{\omega_{cav}^2 + \omega^2}} $$ 

In [ ]:
# example radiation pressure noise
Pcav = 1e6 # [W] cavity power
lambduh = 1064e-9  # [m] laser wavelength
w_0 = 2 *np.pi * const.c / lambduh
m_mirror = 40 # [kg] mirror mass
F_RP = (2 / const.c) * np.sqrt(Pcav * const.hbar * w_0 / (1 + (w/(2*np.pi*f_cav))**2))
F_RP *= np.ones_like(w)

x_shot = 1e-20 * np.ones_like(w) * (1 + (w/(2*np.pi*f_cav)))

err = (F_RP * mag_pend)**2 + (x_shot)**2
err = np.sqrt(err)
err *= mag_clg

fig,ax = plt.subplots(1, figsize=(13,7), sharex='col')
ax.loglog(f, F_RP/m_mirror/w**2, label='RP noise')
ax.loglog(f, x_shot, label='shot noise')
ax.loglog(f, err, label='Closed-Loop Error Signal')

ax.set_ylim([1e-23, 1e-16])
ax.legend()
ax.set_xlabel(r'Frequency [Hz]')
ax.set_ylabel(r'Noise [m/$\sqrt{\rm Hz}$]')
ax.set_title('Sensitivity of the Interferometer')

plt.show()

## Range Analysis

## Stability Analysis

## Optimal Feedback

### Definition of the Cost Function

### Variation of the Parameters


### Multi-Dimensional Search for optimum controller